In [17]:
# Built in python abstract class
from abc import ABC, abstractmethod

import requests
import logging

In [5]:
logger = logging.getLogger(__name__)

In [6]:
logging.basicConfig(level=logging.INFO)

In [2]:
print(requests.get("https://www.mercadobitcoin.net/api/BTC/day-summary/2021/6/21").json())

{'date': '2021-06-21', 'opening': 182018.62236, 'closing': 167597.94809, 'lowest': 158000, 'highest': 182018.62236, 'volume': '47640150.72831583', 'quantity': '286.48516078', 'amount': 19363, 'avg_price': 166291.86167481}


In [19]:
class MercadoBitcoinApi(ABC):

    def __init__(self, coin: str) -> None:
        self.coin = coin
        self.base_endpoint = "https://www.mercadobitcoin.net/api"

    @abstractmethod
    def _get_endpoint(self) -> str:
        pass
    
    def get_data(self) -> dict:
        '''
            Get data form an endpoint
            Raises HTTPError exception if reponse failed
            :return dict: dictionary from endpoint json response
        '''
        endpoint = self._get_endpoint()
        logger.info(f"Getting data from endpoint: {endpoint}")
        response = requests.get(endpoint)
        response.raise_for_status()
        return response.json()


In [22]:
class BtcApi(MercadoBitcoinApi):

    def _get_endpoint(self) -> str:
        '''
            Returns endpoint for coin day summary.
        '''
        return f"{self.base_endpoint}/{self.coin}/day-summary/2021/6/21"

In [23]:
print(BtcApi(coin="BTC").get_data())

INFO:__main__:Getting data from endpoint: https://www.mercadobitcoin.net/api/BTC/day-summary/2021/6/21


{'date': '2021-06-21', 'opening': 182018.62236, 'closing': 167597.94809, 'lowest': 158000, 'highest': 182018.62236, 'volume': '47640150.72831583', 'quantity': '286.48516078', 'amount': 19363, 'avg_price': 166291.86167481}


In [12]:
print(MercadoBitcoinApi(coin="LTC").get_data())

INFO:__main__:Getting data from endpoint: https://www.mercadobitcoin.net/api/LTC/day-summary/2021/6/21


{'date': '2021-06-21', 'opening': 783.42982, 'closing': 663.7, 'lowest': 608.32, 'highest': 784.8656, 'volume': '3378975.43760124', 'quantity': '4944.79127364', 'amount': 3976, 'avg_price': 683.34035768}
